In [1]:
#extend path for loading modules
import sys
import os
sys.path.append(os.path.abspath( '../src/models'))
from custom_functions import loss_custom, hyperpar_search
import pandas as pd
import os
from sklearn.model_selection import train_test_split

import lightgbm as lgb

In [2]:
x_train_total = pd.read_csv('../data/raw/x_train_T9QMMVq.csv')
y_train_total = pd.read_csv('../data/raw/y_train_R0MqWmu.csv')

x_train, x_test, y_train, y_test = train_test_split(x_train_total, y_train_total, test_size=0.2, random_state=22)

x_train = x_train.drop(columns='ID',axis=1)
y_train = y_train.drop(columns='ID',axis=1)

x_test = x_test.drop(columns='ID',axis=1)
y_test = y_test.drop(columns='ID',axis=1)



In [3]:
# Define my model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor
import time

model = RandomForestRegressor(n_jobs=-1)
model_lgbm = LGBMRegressor(n_jobs=-1)
# Set hyperparameter range
param_dist = {
    'n_estimators': [50, 100],  # Lista discreta de valores
    'max_depth': [10, 20]} #, 30, 40, None]}  # Valores posibles incluyendo None
#    'min_samples_split': range(2, 20),  # Rango de 2 a 19
#    'min_samples_leaf': range(1, 10),   # Rango de 1 a 9
#    'max_features': ['sqrt', 'log2', None]  # Estrategias de selección de características
#}

rand_search = RandomizedSearchCV(estimator=model, 
                                 param_distributions=param_dist,
                                cv=5,n_iter=10,n_jobs=-1)


start_fit = time.perf_counter()
model.fit(x_train, y_train)
end_fit = time.perf_counter()
print(f'El fit de RandomForestRegressor tarda {end_fit-start_fit} segundos')

lgbm = lgb.LGBMRegressor(random_state=42, n_jobs=-1)
multioutput_lgbm = MultiOutputRegressor(lgbm)
start_fit = time.perf_counter()
multioutput_lgbm.fit(x_train,y_train)
end_fit = time.perf_counter()
print(f'El fit de LGBM  tarda {end_fit-start_fit} segundos')


El fit de RandomForestRegressor tarda 44.966553102000034 segundos
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003900 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3315
[LightGBM] [Info] Number of data points in the train set: 162346, number of used features: 13
[LightGBM] [Info] Start training from score 0.270867
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003977 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3315
[LightGBM] [Info] Number of data points in the train set: 162346, number of used features: 13
[LightGBM] [Info] Start training from score 0.077333
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004055 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3315
[LightGBM] [Info] Number of data points in the train

In [5]:
# Test both models
from sklearn.model_selection import cross_validate
cv_randfor = cross_validate(model, x_test, y_test, cv=5)
cv_lgbm = cross_validate(multioutput_lgbm, x_test, y_test, cv=5)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.025737 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3315
[LightGBM] [Info] Number of data points in the train set: 32469, number of used features: 13
[LightGBM] [Info] Start training from score 0.272737
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000503 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3315
[LightGBM] [Info] Number of data points in the train set: 32469, number of used features: 13
[LightGBM] [Info] Start training from score 0.076184
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000881 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3315
[LightGBM] [Info] Number of data points in the train set

In [10]:
print(cv_randfor['test_score'].mean())
print(cv_lgbm['test_score'].mean())

0.9798088878210602
0.9685048672960601
